In [ ]:
!pip install -r requirements.txt
!pip install git+https://github.com/openai/whisper.git 
!pip install yt-dlp
!pip install moviepy --upgrade
!apt-get update
!apt install imagemagick -y

In [1]:
# remove line 88 of vim ~/../etc/ImageMagick-6/policy.xml
!sed -i '88d' ~/../etc/ImageMagick-6/policy.xml 

In [3]:
## Imports
from __future__ import unicode_literals
from yt_dlp import YoutubeDL
import yt_dlp
from IPython.display import Video
import whisper
import cv2
import pandas as pd
from moviepy.editor import VideoFileClip
import moviepy.editor as mp
from IPython.display import display, Markdown
from moviepy.editor import *
from moviepy.video.tools.subtitles import SubtitlesClip

## Declare relevant variables

# options for youtube download to ensure we get the right audio file
ydl_opts = {
    'format': 'mp3/bestaudio/best',
    'outtmpl':'t/audio.mp3',
    '-k':True,
    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
    }]
}
# options for youtube video to get right video file for final output
opts2 = {'format': 'mp4/bestvideo/best','outtmpl':'video.mp4'}

# Youtube URL
URL = 'https://youtu.be/MEJJh7tVr4E'

def subtitle_video(download, url, opts, opts_, model_type, audio_file, input_file, output, uploaded_vid = None):
#     params:
#     download: Default is True. This tells your function if you are downloading a youtube video
#     url: URL of youtube video
#     opts: audio youtube-dl options
#     opts_: video youtube-dl options    
#     model_type: which pretrained model to download. options are ['tiny', 'small', 'base', 'medium','large']
#     audio_file: path to extracted audio file
#     input_file: path to video file
#     output: destination of final output
#     uploaded_vid: path to uploaded video file if download is False
#     
#--------------------------------------------------------
#   Returns: annotated video with translated captions into english

    URLS = [url]
    if download:
        with yt_dlp.YoutubeDL(opts) as ydl:
            ydl.download(url)
        with YoutubeDL(opts_) as ydl:
            ydl.download(URLS)
    else:
        my_clip = mp.VideoFileClip(uploaded_vid)
        my_clip.audio.write_audiofile('t/audio.mp3')

    model = whisper.load_model(model_type)
    result = model.transcribe("t/audio.mp3", task = 'translate')
    dict1 = {'start':[], 'end':[], 'text':[]}
    for i in result['segments']:
        dict1['start'].append(i['start'])
        dict1['end'].append(i['end'])
        dict1['text'].append(i['text'])
    df = pd.DataFrame.from_dict(dict1)
    generator = lambda txt: TextClip(txt, font='NimbusMonoPS-Bold', fontsize=24, color='white')
    subs = tuple(zip(tuple(zip(df['start'].values, df['end'].values)), df['text'].values))
    subtitles = SubtitlesClip(subs, generator)
    if download:
        video = VideoFileClip("video.mp4")
        final = CompositeVideoClip([video, subtitles.set_pos(('center','bottom'))])
        final.write_videofile(output, fps=video.fps, remove_temp=True, codec="libx264", audio_codec="aac")
    else:
        video = VideoFileClip(uploaded_vid)
        final = CompositeVideoClip([video, subtitles.set_pos(('center','bottom'))])
        final.write_videofile(output, fps=video.fps, remove_temp=True, codec="libx264", audio_codec="aac")




MoviePy - Writing audio in t/audio.mp3


MoviePy - Done.
Moviepy - Building video output2.mp4.
MoviePy - Writing audio in output2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output2.mp4



Moviepy - Done !
Moviepy - video ready output2.mp4


In [ ]:
subtitle_video(download=False, uploaded_vid=None, url=URL, opts=ydl_opts, opts_={
               'format': 'mp4/bestvideo/best', 'outtmpl': 'video.mp4'}, model_type='large', audio_file="t/audio.mp3", input_file='video.mp4', output='output2.mp4')


<video controls src="output.mp4" />